# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data1 = data.groupby(['CustomerID','ProductName']).agg({"Quantity":"sum"})
data1.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [18]:
data2 = pd.pivot_table(data1, values='Quantity', index='ProductName', columns='CustomerID')
data2 = data2.fillna(value=0)

data2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pd.DataFrame(1/(1+squareform(pdist(data2.T, metric='euclidean'))),
                         index=data2.columns, columns=data2.columns)
distances.head(20)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
464,0.082709,0.075255,0.080350,0.078187,0.078711,1.000000,0.074798,0.084626,0.080070,0.078187,...,0.004816,0.004687,0.004416,0.005038,0.004308,0.004533,0.004578,0.004363,0.004167,0.004347
477,0.074573,0.075956,0.076923,0.075025,0.075025,0.074798,1.000000,0.078977,0.075720,0.074573,...,0.004823,0.004688,0.004426,0.005037,0.004321,0.004534,0.004589,0.004366,0.004168,0.004338
639,0.083020,0.076435,0.080634,0.082403,0.082403,0.084626,0.078977,1.000000,0.085983,0.083020,...,0.004816,0.004682,0.004412,0.005035,0.004308,0.004526,0.004581,0.004363,0.004176,0.004335
649,0.081503,0.077674,0.082100,0.077171,0.078187,0.080070,0.075720,0.085983,1.000000,0.079792,...,0.004810,0.004676,0.004414,0.005035,0.004314,0.004528,0.004585,0.004369,0.004172,0.004329


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similar=distances[33].sort_values(ascending=False)[1:6]
similar=similar.index.to_list()
similar

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
data5 = data1.loc[similar]
data5.head()

Quantity
CustomerID ProductName                         
264        Apricots - Halves                  1
           Apricots Fresh                     1
           Bacardi Breezer - Tropical         1
           Bagel - Plain                      1
           Banana - Leaves                    1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
data6 = data5.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values('Quantity',ascending=False)
data6.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
#data7 = data6.merge(data2, on='ProductName',how='left')
#data7 = data7[data7[33]==0][0:5]
#res = [i for i in data7.index]
#res

In [11]:
data7 = pd.merge(left=data6,right=data2, how='left', left_on='ProductName', right_on='ProductName')
data7 = data7[data7[33]==0][0:5]
#data7.index
res = [i for i in data7.index]
data7.index.to_list()

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [12]:
def recomendation(custID):
    recom={}
    for c in custID:
        similar = distances[c].sort_values(ascending=False)[1:6]
        similar=similar.index.to_list()
        dat5 = data1.loc[similar]    
        dat6 = dat5.groupby("ProductName").agg({'Quantity':'sum'}).sort_values(by="Quantity",ascending=False)
        dat7 = dat6.merge(data2,on="ProductName",how="right")
        dat7 = dat7[dat7[c]==0][0:5]
        res = [i for i in dat7.index]
        recom[c]=res
    return recom
# recomendation(list(set(data["CustomerID"].values)))
# Muestra de 20 customers, si ponemos una lista de todos tarda porque son muchos.
recomendation(list(set(data["CustomerID"].values))[0:20])

{83973: ['Anchovy Paste - 56 G Tube',
  'Appetizer - Mini Egg Roll, Shrimp',
  'Apricots - Halves',
  'Apricots Fresh',
  'Arizona - Green Tea'],
 59399: ['Anchovy Paste - 56 G Tube',
  'Appetizer - Mini Egg Roll, Shrimp',
  'Appetizer - Mushroom Tart',
  'Appetizer - Sausage Rolls',
  'Apricots - Dried'],
 92168: ['Anchovy Paste - 56 G Tube',
  'Appetizer - Mini Egg Roll, Shrimp',
  'Appetizer - Sausage Rolls',
  'Apricots - Dried',
  'Apricots - Halves'],
 49159: ['Anchovy Paste - 56 G Tube',
  'Appetizer - Mini Egg Roll, Shrimp',
  'Appetizer - Mushroom Tart',
  'Appetizer - Sausage Rolls',
  'Apricots Fresh'],
 18441: ['Anchovy Paste - 56 G Tube',
  'Appetizer - Mini Egg Roll, Shrimp',
  'Appetizer - Mushroom Tart',
  'Appetizer - Sausage Rolls',
  'Apricots - Dried'],
 22536: ['Appetizer - Mini Egg Roll, Shrimp',
  'Appetizer - Mushroom Tart',
  'Apricots - Dried',
  'Apricots Fresh',
  'Arizona - Green Tea'],
 86028: ['Anchovy Paste - 56 G Tube',
  'Appetizer - Mini Egg Roll, Shr

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [13]:
data9 = pd.DataFrame(recomendation(list(set(data["CustomerID"].values)))).T
data9.head()

,0,1,2,3,4
83973,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Apricots - Halves,Apricots Fresh,Arizona - Green Tea
59399,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
92168,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves
49159,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots Fresh
18441,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [19]:
distances = pd.DataFrame(1/(1+squareform(pdist(data2.T, metric='jaccard'))),
                         index=data2.columns, columns=data2.columns)
def recomendation(custID):
    recom={}
    for c in custID:
        similar = distances[c].sort_values(ascending=False)[1:6]
        similar=similar.index
        dat5 = data1.loc[similar]    
        dat6 = dat5.groupby("ProductName").agg({'Quantity':'sum'}).sort_values(by="Quantity",ascending=False)
        dat7 = dat6.merge(data2,on="ProductName",how="right")
        dat7 = dat7[dat7[c]==0][0:5]
        res = [i for i in dat7.index]
        recom[c]=res
    return recom
data10 = pd.DataFrame(recomendation(list(set(data["CustomerID"].values)))).T
data10.head()

,0,1,2,3,4
83973,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Apricots - Halves,Apricots Fresh,Arizona - Green Tea
59399,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
92168,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves
49159,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots Fresh
18441,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried


In [21]:
distances = pd.DataFrame(1/(1+squareform(pdist(data2.T, metric='correlation'))),
                         index=data2.columns, columns=data2.columns)
def recomendation(custID):
    recom={}
    for c in custID:
        similar = distances[c].sort_values(ascending=False)[1:6]
        similar=similar.index.to_list()
        dat5 = data1.loc[similar]    
        dat6 = dat5.groupby("ProductName").agg({'Quantity':'sum'}).sort_values(by="Quantity",ascending=False)
        dat7 = dat6.merge(data2,on="ProductName",how="right")
        dat7 = dat7[dat7[c]==0][0:5]
        res = [i for i in dat7.index]
        recom[c]=res
    return recom
data10 = pd.DataFrame(recomendation(list(set(data["CustomerID"].values)))).T
data10.head()

,0,1,2,3,4
83973,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Apricots - Halves,Apricots Fresh,Arizona - Green Tea
59399,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
92168,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves
49159,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots Fresh
18441,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried


In [17]:
distances = pd.DataFrame(1/(1+squareform(pdist(data2.T, metric='cityblock'))),
                         index=data2.columns, columns=data2.columns)
data10 = pd.DataFrame(recomendation(list(set(data["CustomerID"].values)))).T
data10.head()

,0,1,2,3,4
83973,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Apricots - Halves,Apricots Fresh,Arizona - Green Tea
59399,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
92168,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves
49159,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots Fresh
18441,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
